In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext

import pandas as pd

In [ ]:
spark = SparkSession\
            .builder\
            .master("spark://spark-master:7077")\
            .config("spark.sql.repl.eagerEval.enabled", "true")\
            .config('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener')\
            .config('spark.driver.extraClassPath', '/opt/bitnami/python/lib/python3.6/site-packages/sparkmonitor/listener_2.12.jar')\
            .getOrCreate()

In [ ]:
pdf = pd.read_json("arxiv-metadata-oai-snapshot.json", lines=True, nrows=100)
df = spark.createDataFrame(pdf)

In [ ]:
df.registerTempTable('arxiv')

In [ ]:
df_sql_count = spark.sql("SELECT id, SIZE(SPLIT(abstract, ' ')) AS wordcount FROM arxiv")

In [ ]:
df_sql_count

In [ ]:
df_sql_count.registerTempTable("df_sql_count")

In [ ]:
df_sql_count_groupby = spark.sql("SELECT wordcount, COUNT(id) AS count FROM df_sql_count GROUP BY wordcount")

In [ ]:
df_sql_count_groupby.registerTempTable("df_sql_count_groupby")

In [ ]:
df_sql_count_groupby_longest = spark.sql("SELECT * FROM df_sql_count_groupby ORDER BY count DESC LIMIT 20")

In [ ]:
df_sql_count_groupby_longest

In [ ]:
spark.stop()